<center><h2>ALTeGraD 2023<br>Lab Session 4: NLP Frameworks - RoBERTa</h2> 07 / 11 / 2023<br> Dr. G. Shang, H. Abdine<br><br>


<b>Student name:</b> Halvard Bariller

</center>

In this lab you will learn how to use Fairseq and HuggingFace transformers - The most used libraries by researchers and developers  and finetune language models - to finetune a pretrained French language model ($RoBERTa_{small}^{fr}$) on the sentiment analysis dataset CLS_Books where each review is labeled as positive or negative and finetune a variant of BLOOM on a question/answer dataset.

## <b>Preparing the environment and installing libraries, models and data for Part 1 and Part 2</b>

In this section, we will setup the environment on Google Colab (first cell), download the pretraind model (second cell) and the finetuning dataset (third cell). In case you are using your personal computer maket sure to:

1- Use Ubuntu (or any similar linux distribution) or MacOS. <b> P.S. In case you have Windows, please use Google Colab. </b>

2- <b>Use Anaconda</b> and create new environment if you already installed Fairseq since we will be using a slightly modified version of this library.

In [ ]:
!mkdir altegrad.lab3 && cd altegrad.lab3 && mkdir libs
%cd altegrad.lab3/libs
!git clone https://github.com/hadi-abdine/fairseq
!pip install git+https://github.com/hadi-abdine/fairseq
!git clone https://github.com/huggingface/transformers.git
!pip install git+https://github.com/huggingface/transformers.git
!pip install datasets
!pip install evaluate
!pip install sentencepiece
!pip install tensorboardX
%load_ext tensorboard

/content/altegrad.lab3/libs
Cloning into 'fairseq'...
remote: Enumerating objects: 19737, done.
remote: Total 19737 (delta 0), reused 0 (delta 0), pack-reused 19737
Receiving objects: 100% (19737/19737), 18.58 MiB | 25.09 MiB/s, done.
Resolving deltas: 100% (14323/14323), done.
  Cloning https://github.com/hadi-abdine/fairseq to /tmp/pip-req-build-r86hgg6q
  Running command git clone --filter=blob:none --quiet https://github.com/hadi-abdine/fairseq /tmp/pip-req-build-r86hgg6q
  Resolved https://github.com/hadi-abdine/fairseq to commit acdd05d6c5c099d7573e199e340df938b5594e99
  Running command git submodule update --init --recursive -q
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4

In [ ]:
!cd .. && mkdir models
%cd ../models

!wget -c 'https://nuage.lix.polytechnique.fr/index.php/s/E4otcD7B9jm2AWx/download/RoBERTa_small_fr.zip' -O "model_fairseq.zip"
!unzip model_fairseq.zip
!rm model_fairseq.zip
!rm -rf __MACOSX/

!wget -c "https://nuage.lix.polytechnique.fr/index.php/s/yYQjg9XWekttG5j/download/RoBERTa_small_fr_HuggingFace.zip" -O "model_huggingface.zip"
!unzip model_huggingface.zip
!rm model_huggingface.zip
!rm -rf __MACOSX/

/content/altegrad.lab3/models
--2023-11-14 13:16:52--  https://nuage.lix.polytechnique.fr/index.php/s/E4otcD7B9jm2AWx/download/RoBERTa_small_fr.zip
Resolving nuage.lix.polytechnique.fr (nuage.lix.polytechnique.fr)... 193.55.176.11
Connecting to nuage.lix.polytechnique.fr (nuage.lix.polytechnique.fr)|193.55.176.11|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 264887103 (253M) [application/zip]
Saving to: ‘model_fairseq.zip’

model_fairseq.zip   100%[===================>] 252.62M  18.9MB/s    in 14s     

2023-11-14 13:17:07 (17.9 MB/s) - ‘model_fairseq.zip’ saved [264887103/264887103]

Archive:  model_fairseq.zip
   creating: RoBERTa_small_fr/
  inflating: RoBERTa_small_fr/model.pt  
  inflating: __MACOSX/RoBERTa_small_fr/._model.pt  
  inflating: RoBERTa_small_fr/sentencepiece.bpe.model  
  inflating: RoBERTa_small_fr/dict.txt  
--2023-11-14 13:17:19--  https://nuage.lix.polytechnique.fr/index.php/s/yYQjg9XWekttG5j/download/RoBERTa_small_fr_HuggingFace.zip
R

In [ ]:
!cd .. && mkdir data
%cd ../data
!wget -c "https://nuage.lix.polytechnique.fr/index.php/s/EBHqfR776oCE2Nj/download/cls.books.zip" -O "cls.books.zip"
!unzip cls.books.zip
!rm cls.books.zip
!rm -rf __MACOSX/
!mkdir cls.books-json
%cd ..

/content/altegrad.lab3/data
--2023-11-14 13:17:25--  https://nuage.lix.polytechnique.fr/index.php/s/EBHqfR776oCE2Nj/download/cls.books.zip
Resolving nuage.lix.polytechnique.fr (nuage.lix.polytechnique.fr)... 193.55.176.11
Connecting to nuage.lix.polytechnique.fr (nuage.lix.polytechnique.fr)|193.55.176.11|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 955190 (933K) [application/zip]
Saving to: ‘cls.books.zip’

cls.books.zip       100%[===================>] 932.80K  1.48MB/s    in 0.6s    

2023-11-14 13:17:26 (1.48 MB/s) - ‘cls.books.zip’ saved [955190/955190]

Archive:  cls.books.zip
   creating: cls.books/
  inflating: cls.books/valid.label   
  inflating: cls.books/test.review   
  inflating: cls.books/valid.review  
  inflating: cls.books/train.review  
  inflating: cls.books/train.label   
  inflating: cls.books/test.label    
/content/altegrad.lab3


# <b>Part 1: Fairseq</b>

In the first part of this lab, you will finetune the given model on model on CLS_Books dataset using <b>Fairseq</b> by following these steps:<br>

 1- <b>Tokenize the reviews</b> (Train, Valid and Test) using trained sentencepiece tokenizer provided alongside the pretrained model.[using sentencepiece library and setting the parameter <b>out_type=str</b> in the encode function].<br>
 2- <b>Binarize the tokenized reviews and their labels</b> using the preprocess python script provided in Fairseq.<br>
 3- <b>Fintune the pretrained $RoBERTa_{small}^{fr}$ model</b> using the train python script provided in Fairseq.<br>

 Finally, you will finish the first part by training a random $RoBERTa_{small}^{fr}$ model on the CLS_Books dataset and compare the results against the pretrained model while <b>visualizing the accuracies on tensorboard</b>.

## <b> Number of parameters of the model</b>

In this section you have to compute the number of parameters of $RoBERTa_{small}^{fr}$ using PyTorch (Only the base model with out the head). (<b>Hint:</b> you can check the architecture of the model using model['model'])

In [ ]:
import torch

model = torch.load("models/RoBERTa_small_fr/model.pt")


In [ ]:
model["model"].keys()

odict_keys(['encoder.sentence_encoder.version', 'encoder.sentence_encoder.embed_tokens.weight', 'encoder.sentence_encoder.embed_positions.weight', 'encoder.sentence_encoder.layernorm_embedding.weight', 'encoder.sentence_encoder.layernorm_embedding.bias', 'encoder.sentence_encoder.layers.0.self_attn.k_proj.weight', 'encoder.sentence_encoder.layers.0.self_attn.k_proj.bias', 'encoder.sentence_encoder.layers.0.self_attn.v_proj.weight', 'encoder.sentence_encoder.layers.0.self_attn.v_proj.bias', 'encoder.sentence_encoder.layers.0.self_attn.q_proj.weight', 'encoder.sentence_encoder.layers.0.self_attn.q_proj.bias', 'encoder.sentence_encoder.layers.0.self_attn.out_proj.weight', 'encoder.sentence_encoder.layers.0.self_attn.out_proj.bias', 'encoder.sentence_encoder.layers.0.self_attn_layer_norm.weight', 'encoder.sentence_encoder.layers.0.self_attn_layer_norm.bias', 'encoder.sentence_encoder.layers.0.fc1.weight', 'encoder.sentence_encoder.layers.0.fc1.bias', 'encoder.sentence_encoder.layers.0.fc2.

In [ ]:
param_not_to_count = ['encoder.sentence_encoder.version', 'encoder.lm_head.weight', 'encoder.lm_head.bias',
                      'encoder.lm_head.dense.weight', 'encoder.lm_head.dense.bias',
                      'encoder.lm_head.layer_norm.weight', 'encoder.lm_head.layer_norm.bias']

# param_not_to_count = []
params = 0

for param in model["model"]:
  if param not in param_not_to_count:
    params += model["model"][str(param)].numel()

In [ ]:
print("The total number of parameters: ",  params )

The total number of parameters:  22829056


## <b>Tokenizing the reviews</b>

In this section we will tokenize the finetuning dataset using sentenpiece tokenizer. We have three splits in our datase: train valid and test sets.

In this task you have to use the trained sentencepiece tokenizer (RoBERTa_small_fr/sentencepiece.bpe.model) to tokenize the three files <b>train.review</b>, <b>valid.review</b> and <b>test.review</b> and output the three files <b>train.spm.review</b>, <b>valid.spm.review</b> and <b>test.spm.review</b> containing the tokenized reviews.

Documentation: https://github.com/google/sentencepiece#readme

In [ ]:
import sentencepiece as spm
s = spm.SentencePieceProcessor(model_file='models/RoBERTa_small_fr/sentencepiece.bpe.model')

SPLITS=['train', 'test', 'valid']
SENTS="review"

for split in SPLITS:
    with open('data/cls.books/'+split+'.'+SENTS, 'r') as f:
        reviews = f.readlines()
        if split == 'train':
          train_length = len(reviews)
        for i, review in enumerate(reviews):
          reviews[i] = ' '.join(s.encode_as_pieces(review))

        # It should look something like this :
        #▁An ci enne ▁VS ▁Nouvelle ▁version ▁plus

    with open('data/cls.books/'+split+'.spm.'+SENTS, 'w') as f:
        for review in reviews:
          f.write(review+'\n')

In [ ]:
print(reviews[0])

▁Ce ▁livre ▁explique ▁technique ment ▁et ▁de ▁façon ▁très ▁com pré hen sible , ▁même ▁pour ▁des ▁né o phy tes ▁en ▁physique ▁des ▁matériaux , ▁comment ▁et ▁pourquoi ▁la ▁version ▁officielle ▁concernant ▁l ' ef fond re ment ▁des ▁3 ▁tour s ▁du ▁W TC ▁ne ▁tient ▁tout ▁simplement ▁pas ▁de bou t . ▁Et ▁c ' est ▁sans ▁appel ▁! ▁Il ▁constitu e ▁un ▁très ▁bon ▁argument aire ▁scientifique ▁permettant ▁de ▁soutenir ▁l ' hy po th èse ▁la ▁plus ▁vrai sem bla ble ▁pour ▁explique r ▁ce ▁qui ▁s ' est ▁réellement ▁passé ▁à ▁New ▁York , ▁ce ▁jour - là , ▁l ' hy po th èse ▁d ' une ▁dé moli tion ▁cont rô lé e ▁! ▁A ▁lire ▁absolument ▁!


## <b>Binarizing/Preprocessing the finetuning dataset</b>

In this section, you have to binarize the CLS_Books dataset using the <b>fairseq/fairseq_cli/preprocess.py</b> script:

1- Binarize the tokenized reviews and put the output in <b>data/cls-books-bin/input0</b>. Note: Our pretrained model's embedding matrix contains only the embedding of the vocab listed in the dictionary <b>dict.txt</b>. You need to use the dictionary in the binarization of the text to transform the tokens into indices. Also note that we are using Encoder only architecture, so we only have source data.

2- Binarize the labels (train.label, valid.label and test.label files) and put the output in <b>data/cls-books-bin/label</b>.

Documentation: https://fairseq.readthedocs.io/en/latest/command_line_tools.html

In [ ]:
!(python libs/fairseq/fairseq_cli/preprocess.py \
              --only-source \
              --trainpref data/cls.books/train.spm.review \
              --validpref data/cls.books/valid.spm.review \
              --testpref data/cls.books/test.spm.review \
              --destdir data/cls.books-bin/input0 \
              --srcdict models/RoBERTa_small_fr/dict.txt \
              --workers 8)#binarize the tokenized reviews

!(python libs/fairseq/fairseq_cli/preprocess.py \
              --only-source \
              --trainpref data/cls.books/train.label \
              --validpref data/cls.books/valid.label \
              --testpref data/cls.books/test.label \
              --destdir data/cls.books-bin/label \
              --workers 8)# binarize the labels

2023-11-14 13:20:04.396863: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-14 13:20:04.396923: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-14 13:20:04.396960: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-14 13:20:04.405667: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-14 13:20:05.888520: W tensorflow/compiler/

## <b>Finetuning $RoBERTa_{small}^{fr}$</b>

In this section you will use <b>fairseq/fairseq_cli/train.py</b> python script to finetune the pretrained model on the CLS_Books dataset (binarized data) for three different seeds: 0, 1 and 2.

Make sure to use the following hyper-parameters: $\textit{batch size}=8, \textit{max number of epochs}: 5, \textit{optimizer}: Adam, \textit{max learning rate}: 1e-05,  \textit{warm up ratio}: 0.06, \textit{learning rate scheduler}: linear$

In [ ]:
DATA_SET='books'
TASK= "sentence_prediction" # sentence prediction task on fairseq
MODEL='RoBERTa_small_fr'
DATA_PATH= "/content/altegrad.lab3/data/cls.books-bin" 
MODEL_PATH= "/content/altegrad.lab3/models/RoBERTa_small_fr/model.pt" 
MAX_EPOCH= 5 
MAX_SENTENCES= 8 # batch size
MAX_UPDATE= int(MAX_EPOCH * train_length / MAX_SENTENCES) # number of backward propagation steps
LR= 1e-5 
VALID_SUBSET='valid,test' # for simplicity we will validate on both valid and test set, and then pick the value of test set corresponding the best validation score.
METRIC = "accuracy" # use the accuracy metric
NUM_CLASSES= 2 #number of classes
SEEDS=3
CUDA_VISIBLE_DEVICES=0
WARMUP = int(.06 * MAX_UPDATE) # warmup ratio=6% of the whole training

In [ ]:
for SEED in range(SEEDS):
  TENSORBOARD_LOGS= 'tensorboard_logs/'+TASK+'/'+DATA_SET+'/'+MODEL+'_ms'+str(MAX_SENTENCES)+'_mu'+str(MAX_UPDATE)+'_lr'+str(LR)+'_me'+str(MAX_EPOCH)+'/'+str(SEED)
  SAVE_DIR= 'checkpoints/'+TASK+'/'+DATA_SET+'/'+MODEL+'_ms'+str(MAX_SENTENCES)+'_mu'+str(MAX_UPDATE)+'_lr'+str(LR)+'_me'+str(MAX_EPOCH)+'/'+str(SEED)
  !(python libs/fairseq/fairseq_cli/train.py $DATA_PATH \
                --restore-file $MODEL_PATH \
                --batch-size $MAX_SENTENCES \
                --task $TASK \
                --update-freq 1 \
                --seed $SEED \
                --reset-optimizer --reset-dataloader --reset-meters \
                --init-token 0 \
                --separator-token 2 \
                --arch roberta_small \
                --criterion sentence_prediction \
                --num-classes $NUM_CLASSES \
                --weight-decay 0.01 \
                --optimizer adam --adam-betas "(0.9, 0.98)" --adam-eps 1e-08 \
                --maximize-best-checkpoint-metric \
                --best-checkpoint-metric 'accuracy' \
                --save-dir $SAVE_DIR \
                --lr-scheduler polynomial_decay \
                --lr $LR \
                --max-update $MAX_UPDATE \
                --total-num-update $MAX_UPDATE \
                --no-epoch-checkpoints \
                --no-last-checkpoints \
                --tensorboard-logdir $TENSORBOARD_LOGS \
                --log-interval 5 \
                --warmup-updates $WARMUP \
                --max-epoch $MAX_EPOCH \
                --keep-best-checkpoints 1 \
                --max-positions 256 \
                --valid-subset $VALID_SUBSET \
                --shorten-method 'truncate' \
                --no-save \
                --distributed-world-size 1)


2023-11-14 13:20:27.550881: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-14 13:20:27.550941: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-14 13:20:27.550976: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-14 13:20:27.558476: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-14 13:20:29.173326: W tensorflow/compiler/

## <b>Random $RoBERTa_{small}^{fr}$ model training:</b>

In this section you have to finetune a random checkpinf of the model $RoBERTa_{small}^{fr}$ using the same setting as before (<b>Hint:</b> an unexisted model path will not give you an error)

In [ ]:
DATA_SET='books'
TASK= "sentence_prediction" # sentence prediction task on fairseq
MODEL='RoBERTa_small_fr_random'
DATA_PATH= "/content/altegrad.lab3/data/cls.books-bin" 
MODEL_PATH= '/content/' 
MAX_EPOCH= 5 
MAX_SENTENCES= 8 # batch size
MAX_UPDATE= int(MAX_EPOCH * train_length / MAX_SENTENCES) # n_epochs * n_train_examples / total batch size
LR= 1e-5 
VALID_SUBSET='valid,test' # for simplicity we will validate on both valid and test set, and then pick the value of test set corresponding the best validation score.
METRIC = "accuracy" # use the accuracy metric
NUM_CLASSES= 2 #number of classes
SEEDS=3
CUDA_VISIBLE_DEVICES=0
WARMUP = int(.06 * MAX_UPDATE) # warmup ratio=6% of the whole training

In [ ]:
for SEED in range(SEEDS):
  TENSORBOARD_LOGS= 'tensorboard_logs/'+TASK+'/'+DATA_SET+'/'+MODEL+'_ms'+str(MAX_SENTENCES)+'_mu'+str(MAX_UPDATE)+'_lr'+str(LR)+'_me'+str(MAX_EPOCH)+'/'+str(SEED)
  SAVE_DIR= 'checkpoints/'+TASK+'/'+DATA_SET+'/'+MODEL+'_ms'+str(MAX_SENTENCES)+'_mu'+str(MAX_UPDATE)+'_lr'+str(LR)+'_me'+str(MAX_EPOCH)+'/'+str(SEED)
  !(python libs/fairseq/fairseq_cli/train.py $DATA_PATH \
                --restore-file $MODEL_PATH \
                --batch-size $MAX_SENTENCES \
                --task $TASK \
                --update-freq 1 \
                --seed $SEED \
                --reset-optimizer --reset-dataloader --reset-meters \
                --init-token 0 \
                --separator-token 2 \
                --arch roberta_small \
                --criterion sentence_prediction \
                --num-classes $NUM_CLASSES \
                --weight-decay 0.01 \
                --optimizer adam --adam-betas "(0.9, 0.98)" --adam-eps 1e-08 \
                --maximize-best-checkpoint-metric \
                --best-checkpoint-metric 'accuracy' \
                --save-dir $SAVE_DIR \
                --lr-scheduler polynomial_decay \
                --lr $LR \
                --max-update $MAX_UPDATE \
                --total-num-update $MAX_UPDATE \
                --no-epoch-checkpoints \
                --no-last-checkpoints \
                --tensorboard-logdir $TENSORBOARD_LOGS \
                --log-interval 5 \
                --warmup-updates $((6*$MAX_UPDATE/100)) \
                --max-epoch $MAX_EPOCH \
                --keep-best-checkpoints 1 \
                --max-positions 256 \
                --valid-subset $VALID_SUBSET \
                --shorten-method 'truncate' \
                --no-save \
                --distributed-world-size 1)

2023-11-14 13:25:38.642633: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-14 13:25:38.642685: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-14 13:25:38.642719: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-14 13:25:38.650352: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-14 13:25:39.782071: W tensorflow/compiler/

## <b>Tensorboard Visualisation </B>

In the this we will use tensorboard to visualize the training, validation and test accuracies. <b>Include and analyse in you report a screenshot of the test accuracy of the six models</b>.

In [ ]:
%tensorboard --logdir tensorboard_logs

# <b>Part 2: Finetuning $RoBERTa_{small}^{fr}$ using HuggingFace's Transfromers</b>
In this section of the lab, we will finetune a HuggingFace checkpoint of our $RoBERTa_{small}^{fr}$ on the CLS_Books dataset. Like in the first part we will start by downloading the HuggingFace checkpoint and <b>preparing a json format of the CLS_Books dataset</b> (Which is suitable for HuggingFace's checkpoints finetuning using their run_glue script).

## <b>Converting the CLS_Books dataset to json line files</b>

Unlike Fairseq, you do not need to perform tokenization and binarization in Hugging Face transformer library. However, in order to use the implemented script in the transformers library, you need to convert your data to json line files (for each split: train, valid and test)

for instance, each line inside you file will consist of one and one sample only, contaning the review (accessed by the key <i>sentence1</i> and its label, accessed by the key <i>label</i>. Below you can find an example from <i>valid.json</i> file.

Note that these instructions are not valid for all kind of tasks. For other types of tasks (supported in Hugging face) you have to refer to their github for more details.<br>

---------------------------------------------------------------------
<i>
{"sentence1":"Seul ouvrage fran\u00e7ais sur le th\u00e8me Produits Structur\u00e9s \/ fonds \u00e0 formule, il permet de fa\u00e7on p\u00e9dagogique d'appr\u00e9hender parfaitement les m\u00e9canismes financiers utilis\u00e9s. Une r\u00e9f\u00e9rence pour ceux qui veulent comprendre les technicit\u00e9s de base et les raisons de l'engouement des investisseurs sur ces actifs \u00e0 hauteur de plusieurs milliards d'euros.","label":"1"}<br>
{"sentence1":"Livre tr\u00e8s int\u00e9ressant !  mais si comme moi vous cherchez des \"infos\" sur les techniques de sorties et autres \"modes d'emploi\", afin de vivre par vous m\u00eame ce genre d'exp\u00e9rience, c'est pas le bon livre.  \u00e7a ne lui enl\u00e8ve d'ailleurd rien \u00e0 son int\u00earet.","label":"0"}
</i>

---------------------------------------------------------------------



In [ ]:
import json

SPLITS=['train', 'test', 'valid']

for split in SPLITS:
    with open('data/cls.books/'+split+'.review', 'r') as f:
        reviews = f.readlines()
    with open('data/cls.books/'+split+'.label', 'r') as f:
        labels = f.readlines()
    with open('data/cls.books-json/'+split+'.json', 'w') as f:
        #create train.json, valid.json and test.json
        for i, review in enumerate(reviews):
          f.write(json.dumps({'label': labels[i], 'text': review})+'\n')


## <b>Finetuning $RoBERTa_{small}^{fr}$ using the Transformers Library</b>

In order to finrtune the model using HuggingFace, you to use the <b>run_glue.py</b> Python script located in the transformers library. For more details, refer to <a href="https://github.com/huggingface/transformers/tree/main/examples/pytorch/text-classification" target="_blank">the Huggingface/transformers repository on Github</a>. Make sure to use the same hyperparameter as in the first part of this lab.

In [ ]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.7 MB/s eta 0:00:00


In [ ]:
DATA_SET='books'
MODEL='RoBERTa_small_fr_huggingface'
MAX_SENTENCES= 8 # batch size.
LR= 1e-5 #learning rate
MAX_EPOCH= 5 
NUM_CLASSES= 2 
SEEDS= 3
TASK= "sentence_prediction"
CUDA_VISIBLE_DEVICES=0

In [ ]:
for SEED in range(SEEDS):
  SAVE_DIR= 'checkpoints/'+TASK+'/'+DATA_SET+'/'+MODEL+'_ms'+str(MAX_SENTENCES)+'_lr'+str(LR)+'_me'+str(MAX_EPOCH)+'/'+str(SEED)
  !(python libs/transformers/examples/pytorch/text-classification/run_glue.py \
        --model_name_or_path /content/altegrad.lab3/models/RoBERTa_small_fr_HuggingFace \
        --train_file /content/altegrad.lab3/data/cls.books-json/train.json \
        --validation_file /content/altegrad.lab3/data/cls.books-json/valid.json \
        --do_train \
        --do_eval \
        --per_device_train_batch_size $MAX_SENTENCES \
        --num_train_epochs $MAX_EPOCH \
        --max_seq_length 256 \
        --learning_rate $LR \
        --lr_scheduler_type 'linear' \
        --weight_decay 0.01 \
        --adam_beta1 0.9 \
        --adam_beta2 0.98 \
        --adam_epsilon 1e-08 \
        --warmup_ratio 0.06 \
        --seed $SEED \
        --evaluation_strategy steps \
        --eval_steps 225 \
        --logging_steps 5 \
        --logging_dir $SAVE_DIR/valid \
        --metric_for_best_model 'accuracy' \
        --output_dir $SAVE_DIR \
        --overwrite_output_dir )

  !(python libs/transformers/examples/pytorch/text-classification/run_glue.py \
        --model_name_or_path /content/altegrad.lab3/models/RoBERTa_small_fr_HuggingFace \
        --train_file /content/altegrad.lab3/data/cls.books-json/train.json \
        --validation_file /content/altegrad.lab3/data/cls.books-json/test.json \
        --do_train \
        --do_eval \
        --per_device_train_batch_size $MAX_SENTENCES \
        --num_train_epochs $MAX_EPOCH \
        --max_seq_length 256 \
        --learning_rate $LR \
        --lr_scheduler_type 'linear' \
        --weight_decay 0.01 \
        --adam_beta1 0.9 \
        --adam_beta2 0.98 \
        --adam_epsilon 1e-08 \
        --warmup_ratio 0.06 \
        --seed $SEED \
        --evaluation_strategy steps \
        --eval_steps 225 \
        --logging_steps 5 \
        --logging_dir $SAVE_DIR/test \
        --metric_for_best_model 'accuracy' \
        --output_dir $SAVE_DIR \
        --overwrite_output_dir )

2023-11-14 13:38:05.880516: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-14 13:38:05.880569: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-14 13:38:05.880610: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-14 13:38:10.275678: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
11/14/2023 13:38:18 - WARNING - __main__ - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
11/14/2023 13:38:18 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,

In [ ]:
%tensorboard --logdir checkpoints --port 6007